In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch
from torch.utils import data
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
import nltk
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings(action='ignore')

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 1.10.2  Device: cpu


통합된 데이터파일 만들기

In [69]:
df_game=pd.read_csv("../data/all_game.csv")
df_game_s=pd.read_csv("../data//all_game_st.csv",converters={"encoding_summary": lambda x: x.strip("[]").split(", ")})
df_game_u=pd.merge(df_game,df_game_s)
df_game_u=df_game_u.drop(['Unnamed: 0'],axis=1)

CNN모델 클래스

In [84]:
class Model_1(nn.Module):
    def __init__(self):
        super(Model_1, self).__init__()      
        self.conv1 = nn.Conv2d(1, 9, kernel_size=20, padding=0)
        self.conv2 = nn.Conv2d(9, 16, kernel_size=20, padding=0)
        self.conv3 = nn.Conv2d(16, 2, kernel_size=20, padding=0)
        self.fc1 = nn.Linear(1710, 300)
        self.fc2 = nn.Linear(300,50)
        self.fc3=nn.Linear(50,1)
        self.embedding = nn.Embedding(46571,300)
        self.relu = torch.nn.ReLU()
        for m in self.modules():
            if isinstance(m, nn.Linear):
                torch.nn.init.kaiming_uniform_(m.weight.data, nonlinearity='relu')
            if isinstance(m,nn.Conv2d):
                torch.nn.init.kaiming_uniform_(m.weight.data, nonlinearity='relu')
            if isinstance(m,nn.Embedding):
                torch.nn.init.kaiming_uniform_(m.weight.data, nonlinearity='relu')
    def forward(self,x):
        x=self.embedding(x.long())
        x=x.unsqueeze(1)
        x = self.relu(self.conv1(x))
        x = torch.nn.functional.avg_pool2d(x, kernel_size=4,stride=2)
        x=self.relu(self.conv2(x))
        x= torch.nn.functional.avg_pool2d(x,kernel_size=5,stride=2)
        x=self.relu(self.conv3(x))
        x= torch.nn.functional.avg_pool2d(x,kernel_size=3,stride=2)
        x=torch.flatten(x,1)
        x=self.relu(self.fc1(x))
        x=self.relu(self.fc2(x))
        x=self.relu(self.fc3(x))
        return x

이미 훈련시킨 CNN가져오기

In [85]:
cnn_model=Model_1()
state_cnn=torch.load('model.tar',map_location=torch.device('cpu'))
state_cnn.keys()
cnn_model.load_state_dict(state_cnn)

<All keys matched successfully>

랜덤포레스트 + CNN(앙상블 모델이아닌 CNN으로 예측한 칼럼을 이용하여 랜덤포레스트)

CNN예측값 칼럼 만들기

In [72]:
df_game_u.dtypes

title                object
metascore             int64
userscore           float64
genres               object
publisher            object
developer            object
release_date          int64
summary              object
rating               object
platform             object
token_summary        object
encoding_summary     object
dtype: object

In [89]:
cnn_x=list(df_game_u['encoding_summary'].values)
cnn_column=[]
cnt=0
interval=len(cnn_x)/10
with torch.no_grad():
    cnn_model.eval()
    for x in cnn_x:
        y=list(map(int, x))
        minput=torch.tensor(y,dtype=torch.float32)
        outputs=cnn_model(minput.view(1,-1))
        cnn_column.append(outputs.item())
        cnt=cnt+1
        if(cnt%100==0): print(cnt) 
        if(cnt%interval==0):
            print(cnt/interval*10,"% 진행중")

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600


랜덤포레스트를 위한 데이터셋준비

In [97]:
dic_genre={}
for idx,row in df_game_u.iterrows():
    nowg=row['genres']
    nowg=nowg.split('|')
    nowg.pop()
    for x in nowg:
        if(x in dic_genre):
            dic_genre[x]+=1
        else:
            dic_genre[x]=0
lgenres=list(dic_genre.keys())
jgenres = pd.DataFrame(np.zeros((len(df_game_u), len(lgenres))),columns=lgenres)
for i, genre in enumerate(df_game_u.genres):
    k=genre.split('|')
    k.pop()
    k=list(set(k))
    jgenres.loc[i, k] = 1
df_game_r = df_game_u.join(jgenres.add_prefix('genre_'))
df_game_r=df_game_r.drop(['genres','summary','encoding_summary','token_summary'],axis=1)
df_game_r=pd.get_dummies(df_game_r, columns = ['developer','platform','rating','release_date','publisher'])
df_game_r

,title,metascore,userscore,cnn_user_score,genre_Simulation,genre_Virtual,genre_Career,genre_Action Adventure,genre_Sci-Fi,genre_General,...,publisher_souvenir circ.,publisher_stillalive studios,publisher_superflat games,publisher_team ok,publisher_teedoubleuGAMES,publisher_tiny Build GAMES,publisher_tinyBuild,publisher_tobyfox,publisher_ustwo,publisher_worm club
0,Surgeon Simulator: Experience Reality,43,4.40,7.736472,1.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,Transformers: Rise of the Dark Spark,43,4.20,7.910106,0.0,0.0,0.0,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,NBA Live 14,43,2.60,7.704445,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,WWE 2K20,43,1.60,7.189451,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,VR Karts,42,5.60,8.024807,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14614,Road 96,78,6.70,8.177712,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
14615,WRC 9 FIA World Rally Championship,78,6.00,7.896835,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
14616,Shadow Warrior 3,78,6.00,7.243953,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
14617,Blast Brigade vs. the Evil Legion of Dr. Cread,78,6.53,7.517102,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


cnn의 데이터를 이용하지않은 랜덤포레스트 와 이용하는 랜덤포레스트 데이터 나누기

In [98]:
x_train_no_cnn, x_test_no_cnn, y_train_no_cnn, y_test_no_cnn = train_test_split(df_game_r.drop(['title','userscore','cnn_user_score'],axis=1), df_game_r['userscore'], random_state = 0)
x_train_cnn, x_test_cnn, y_train_cnn, y_test_cnn = train_test_split(df_game_r.drop(['title','userscore'],axis=1), df_game_r['userscore'], random_state = 0)

랜덤포레스트 학습 파라미터 60과 65로 나눔

In [99]:
x_train_cnn['cnn_user_score'].describe()

count    10964.000000
mean         7.222914
std          1.016421
min          3.053160
25%          6.935300
50%          7.323657
75%          7.791650
max         13.179913
Name: cnn_user_score, dtype: float64

In [102]:
model_cnn_60=RandomForestRegressor(n_estimators=60)
model_cnn_65=RandomForestRegressor(n_estimators=65)
model_cnn_60.fit(x_train_cnn,y_train_cnn)

RandomForestRegressor(n_estimators=60)

In [103]:
model_cnn_65.fit(x_train_cnn,y_train_cnn)

RandomForestRegressor(n_estimators=65)

In [106]:
model_no_cnn_60=RandomForestRegressor(n_estimators=60)
model_no_cnn_60.fit(x_train_no_cnn,y_train_no_cnn)

RandomForestRegressor(n_estimators=60)

In [108]:
model_no_cnn_65=RandomForestRegressor(n_estimators=65)
model_no_cnn_65.fit(x_train_no_cnn,y_train_no_cnn)

RandomForestRegressor(n_estimators=65)

In [109]:
print(model_cnn_60.score(x_test_cnn,y_test_cnn))
print(model_cnn_65.score(x_test_cnn,y_test_cnn))

0.40815663534986435
0.41362509778541356


In [110]:
print(model_no_cnn_60.score(x_test_no_cnn,y_test_no_cnn))
print(model_no_cnn_65.score(x_test_no_cnn,y_test_no_cnn))

0.4175806293323837
0.4195048879903154


In [111]:
pred=model_cnn_60.predict(x_test_cnn)
mse_cnn_60=np.square(np.subtract(y_test_cnn,pred)).mean()
pred=model_cnn_65.predict(x_test_cnn)
mse_cnn_65=np.square(np.subtract(y_test_cnn,pred)).mean()
pred=model_no_cnn_60.predict(x_test_no_cnn)
mse_no_cnn_60=np.square(np.subtract(y_test_no_cnn,pred)).mean()
pred=model_no_cnn_65.predict(x_test_no_cnn)
mse_no_cnn_65=np.square(np.subtract(y_test_no_cnn,pred)).mean()

In [112]:
print(mse_cnn_60,mse_cnn_65,mse_no_cnn_60,mse_no_cnn_65)

1.03458362170965 1.0250243666606431 1.0181098206877057 1.0147460818839318
